In [1]:
import torch
import torch.utils.data
from tqdm.auto import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
from collections import Counter

/home/ray/miniconda3/envs/IDLSG2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
from networks import pretrained_cvt_13_224
from dataloader import *

In [3]:
model = pretrained_cvt_13_224()
model.eval()
model.to(device)

/home/ray/proj/IDL_11785_Project/networks.py:1508: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(pretrained, map_location='cpu')


ConvolutionalVisionTransformer(
  (stage0): VisionTransformer(
    (patch_embed): ConvEmbed(
      (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(2, 2))
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (conv_proj_q): Sequential(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (rearrage): Rearrange('b c h w -> b (h w) c')
          )
          (conv_proj_k): Sequential(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          

In [8]:
dataset_val, _= build_imagenet_val_dataset(input_size=224)
imagenet_dataloader_val = torch.utils.data.DataLoader(
    dataset=dataset_val,
    batch_size=1,
    num_workers = 0,
)

Transform = 
Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
CenterCrop(size=(224, 224))
ToTensor()
Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
---------------------------
reading from datapath data/imagenet
Number of the class = 1000


In [9]:
for inputs, labels in imagenet_dataloader_val:
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([1, 3, 224, 224])
torch.Size([1])


In [11]:
with torch.no_grad():
    for input, label in imagenet_dataloader_val:
        input, label = input.to('cuda'), label.to('cuda')
        output = model(input)
        print(output.shape)
        break


torch.Size([1000])


In [12]:
def validate(model, dataloader):
    correct_top1 = 0
    total_samples = 0
    accumulated_label = 0
    accumulated_predictions = []

    with torch.no_grad():
        for input, label in tqdm(dataloader):

            input, label = input.to('cuda'), label.to('cuda')

            # Forward pass
            output = model(input)
            output = output.unsqueeze(0)

            # Get top-1 predictions
            _, pred_top1 = output.topk(1, dim=1, largest=True, sorted=True)
            
            curr_label = label.item()
            if curr_label==accumulated_label:
                accumulated_predictions.append(pred_top1.item())
            else:
                counter = Counter(accumulated_predictions)
                _, count = counter.most_common(1)[0]
                total_samples += len(accumulated_predictions)
                correct_top1 += count
                accumulated_label = curr_label
                accumulated_predictions = []
    counter = Counter(accumulated_predictions)
    _, count = counter.most_common(1)[0]
    total_samples += len(accumulated_predictions)
    correct_top1 += count
    # Calculate top-1 accuracy percentage
    top1_accuracy = 100 * correct_top1 / total_samples
    return top1_accuracy

top1_accuracy = validate(model, imagenet_dataloader_val)
print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')

100%|██████████| 50000/50000 [13:57<00:00, 59.68it/s]

Top-1 Accuracy: 81.72%
